In [4]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved.
# Set up custom environment before nearly anything else is imported
# NOTE: this should be the first import (no not reorder)
from maskrcnn_benchmark.utils.env import setup_environment  # noqa F401 isort:skip

import argparse
import os

import torch
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.data import make_data_loader
from maskrcnn_benchmark.engine.inference import inference
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.utils.collect_env import collect_env_info
from maskrcnn_benchmark.utils.comm import synchronize, get_rank
from maskrcnn_benchmark.utils.logger import setup_logger
from maskrcnn_benchmark.utils.miscellaneous import mkdir

num_gpus = int(os.environ["WORLD_SIZE"]) if "WORLD_SIZE" in os.environ else 1
distributed = num_gpus > 1

if distributed:
    torch.cuda.set_device(0)
    torch.distributed.init_process_group(
        backend="nccl", init_method="env://"
    )
    synchronize()

cfg.merge_from_file("/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml")
cfg.merge_from_list(['DATALOADER.NUM_WORKERS', '2', \
                     'DATATYPE', 'bee', \
                     'OUTPUT_DIR', 'training_dir/fcos_kps_ms_training_R_50_FPN_1x_bee1', \
                     'MODEL.WEIGHT', '/home/bsb2144/directpose/tools/training_dir/fcos_kps_ms_training_R_50_FPN_1x_bee1/model_final.pth',\
                     'DATASETS.TEST', "('bee_train_cocostyle',)",\
                     'DATASETS.TRAIN', "('bee_train_cocostyle', )",\
                    ])
cfg.freeze()


save_dir = ""
logger = setup_logger("maskrcnn_benchmark", save_dir, get_rank())
logger.info("Using {} GPUs".format(num_gpus))
logger.info(cfg)

logger.info("Collecting env info (might take some time)")
logger.info("\n" + collect_env_info())

model = build_detection_model(cfg)
model.to(cfg.MODEL.DEVICE)

output_dir = cfg.OUTPUT_DIR
checkpointer = DetectronCheckpointer(cfg, model, save_dir=output_dir)
_ = checkpointer.load(cfg.MODEL.WEIGHT)

iou_types = ("bbox",)
if cfg.MODEL.MASK_ON:
    iou_types = iou_types + ("segm",)
if cfg.MODEL.KEYPOINT_ON:
    iou_types = iou_types + ("keypoints",)
output_folders = [None] * len(cfg.DATASETS.TEST)
dataset_names = cfg.DATASETS.TEST

if cfg.OUTPUT_DIR:
    for idx, dataset_name in enumerate(dataset_names):
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference", dataset_name)
        mkdir(output_folder)
        output_folders[idx] = output_folder
data_loaders_val = make_data_loader(cfg, is_train=False, is_distributed=distributed)

for output_folder, dataset_name, data_loader_val in zip(output_folders, dataset_names, data_loaders_val):
    inference(
        model,
        data_loader_val,
        dataset_name=dataset_name,
        iou_types=iou_types,
        box_only=False if cfg.MODEL.FCOS_ON or cfg.MODEL.RETINANET_ON else cfg.MODEL.RPN_ONLY,
        device=cfg.MODEL.DEVICE,
        expected_results=cfg.TEST.EXPECTED_RESULTS,
        expected_results_sigma_tol=cfg.TEST.EXPECTED_RESULTS_SIGMA_TOL,
        output_folder=output_folder,
        datatype='bee',
    )
    synchronize()


2020-09-07 00:11:33,908 maskrcnn_benchmark INFO: Using 1 GPUs
2020-09-07 00:11:33,908 maskrcnn_benchmark INFO: Using 1 GPUs
2020-09-07 00:11:33,908 maskrcnn_benchmark INFO: Using 1 GPUs
2020-09-07 00:11:33,908 maskrcnn_benchmark INFO: Using 1 GPUs
2020-09-07 00:11:33,912 maskrcnn_benchmark INFO: AMP_VERBOSE: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  NUM_WORKERS: 2
  SIZE_DIVISIBILITY: 32
DATASETS:
  TEST: ('bee_train_cocostyle',)
  TRAIN: ('bee_train_cocostyle',)
DATATYPE: bee
DTYPE: float32
INPUT:
  BRIGHTNESS: 0.0
  CONTRAST: 0.0
  CROP_SIZE: 0
  FLIP_PROB_TRAIN: 0.5
  HUE: 0.0
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_RANGE_TRAIN: (-1, -1)
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  PIXEL_MEAN: [102.9801, 115.9465, 122.7717]
  PIXEL_STD: [1.0, 1.0, 1.0]
  SATURATION: 0.0
  TO_BGR255: True
MODEL:
  BACKBONE:
    CONV_BODY: R-50-FPN-RETINANET
    FREEZE_CONV_BODY_AT: 2
    USE_GN: False
  CLS_AGNOSTIC_BBOX_REG: False
  DEVICE: cuda
  F

2020-09-07 00:11:33,912 maskrcnn_benchmark INFO: AMP_VERBOSE: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  NUM_WORKERS: 2
  SIZE_DIVISIBILITY: 32
DATASETS:
  TEST: ('bee_train_cocostyle',)
  TRAIN: ('bee_train_cocostyle',)
DATATYPE: bee
DTYPE: float32
INPUT:
  BRIGHTNESS: 0.0
  CONTRAST: 0.0
  CROP_SIZE: 0
  FLIP_PROB_TRAIN: 0.5
  HUE: 0.0
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_RANGE_TRAIN: (-1, -1)
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  PIXEL_MEAN: [102.9801, 115.9465, 122.7717]
  PIXEL_STD: [1.0, 1.0, 1.0]
  SATURATION: 0.0
  TO_BGR255: True
MODEL:
  BACKBONE:
    CONV_BODY: R-50-FPN-RETINANET
    FREEZE_CONV_BODY_AT: 2
    USE_GN: False
  CLS_AGNOSTIC_BBOX_REG: False
  DEVICE: cuda
  FBNET:
    ARCH: default
    ARCH_DEF: 
    BN_TYPE: bn
    DET_HEAD_BLOCKS: []
    DET_HEAD_LAST_SCALE: 1.0
    DET_HEAD_STRIDE: 0
    DW_CONV_SKIP_BN: True
    DW_CONV_SKIP_RELU: True
    KPTS_HEAD_BLOCKS: []
    KPTS_HEAD_LAST_SCALE: 0.0
    KPTS

2020-09-07 00:11:33,927 maskrcnn_benchmark INFO: Collecting env info (might take some time)
2020-09-07 00:11:33,927 maskrcnn_benchmark INFO: Collecting env info (might take some time)
2020-09-07 00:11:33,927 maskrcnn_benchmark INFO: Collecting env info (might take some time)
2020-09-07 00:11:33,927 maskrcnn_benchmark INFO: Collecting env info (might take some time)
2020-09-07 00:11:38,598 maskrcnn_benchmark INFO: 
PyTorch version: 1.1.0
Is debug build: No
CUDA used to build PyTorch: 9.0.176

OS: CentOS Linux 7 (Core)
GCC version: (GCC) 5.2.0
CMake version: Could not collect

Python version: 3.6
Is CUDA available: Yes
CUDA runtime version: 10.1.168
GPU models and configuration: GPU 0: GeForce GTX 1080 Ti
Nvidia driver version: 418.87.00
cuDNN version: Could not collect

Versions of relevant libraries:
[pip3] numpy==1.19.1
[pip3] torch==1.1.0
[pip3] torchvision==0.2.2
[conda] blas                      1.0                         mkl  
[conda] mkl                       2020.2             

2020-09-07 00:11:39,526 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.bn1.running_var                           loaded from backbone.body.bn1.running_var                           of shape (64,)
2020-09-07 00:11:39,526 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.bn1.running_var                           loaded from backbone.body.bn1.running_var                           of shape (64,)
2020-09-07 00:11:39,526 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.bn1.running_var                           loaded from backbone.body.bn1.running_var                           of shape (64,)
2020-09-07 00:11:39,528 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.bn1.weight                                loaded from backbone.body.bn1.weight                                of shape (64,)
2020-09-07 00:11:39,528 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.bn1.weight                                loaded from b

2020-09-07 00:11:39,550 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.0.bn2.running_mean                 loaded from backbone.body.layer1.0.bn2.running_mean                 of shape (64,)
2020-09-07 00:11:39,550 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.0.bn2.running_mean                 loaded from backbone.body.layer1.0.bn2.running_mean                 of shape (64,)
2020-09-07 00:11:39,550 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.0.bn2.running_mean                 loaded from backbone.body.layer1.0.bn2.running_mean                 of shape (64,)
2020-09-07 00:11:39,550 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.0.bn2.running_mean                 loaded from backbone.body.layer1.0.bn2.running_mean                 of shape (64,)
2020-09-07 00:11:39,552 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.0.bn2.running_var                  loaded from b

...

2020-09-07 00:11:40,787 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.sample_features_conv.1.bias                    loaded from rpn.head.sample_features_conv.1.bias                    of shape (64,)
2020-09-07 00:11:40,787 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.sample_features_conv.1.bias                    loaded from rpn.head.sample_features_conv.1.bias                    of shape (64,)
2020-09-07 00:11:40,787 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.sample_features_conv.1.bias                    loaded from rpn.head.sample_features_conv.1.bias                    of shape (64,)
2020-09-07 00:11:40,787 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.sample_features_conv.1.bias                    loaded from rpn.head.sample_features_conv.1.bias                    of shape (64,)
2020-09-07 00:11:40,790 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.sample_features_conv.1.weight                  loaded from r



  0%|          | 0/17 [00:00<?, ?it/s]

  6%|▌         | 1/17 [00:03<00:54,  3.39s/it]

 12%|█▏        | 2/17 [00:03<00:29,  1.98s/it]

 18%|█▊        | 3/17 [00:05<00:25,  1.84s/it]

 24%|██▎       | 4/17 [00:06<00:19,  1.52s/it]

 29%|██▉       | 5/17 [00:07<00:18,  1.52s/it]

 35%|███▌      | 6/17 [00:08<00:15,  1.37s/it]

 41%|████      | 7/17 [00:09<00:13,  1.39s/it]

 47%|████▋     | 8/17 [00:10<00:11,  1.29s/it]

 53%|█████▎    | 9/17 [00:11<00:10,  1.32s/it]

 59%|█████▉    | 10/17 [00:12<00:08,  1.25s/it]

 65%|██████▍   | 11/17 [00:14<00:07,  1.28s/it]

 71%|███████   | 12/17 [00:14<00:06,  1.22s/it]

 76%|███████▋  | 13/17 [00:16<00:05,  1.26s/it]

 82%|████████▏ | 14/17 [00:16<00:03,  1.21s/it]

 88%|████████▊ | 15/17 [00:18<00:02,  1.22s/it]

 94%|█████████▍| 16/17 [00:18<00:01,  1.18s/it]

100%|██████████| 17/17 [00:19<00:00,  1.14s/it]



2020-09-07 00:12:00,587 maskrcnn_benchmark.inference INFO: Total run time: 0:00:19.568536 (0.14495212060433846 s / img per device, on 1 devices)
2020-09-07 00:12:00,587 maskrcnn_benchmark.inference INFO: Total run time: 0:00:19.568536 (0.14495212060433846 s / img per device, on 1 devices)
2020-09-07 00:12:00,587 maskrcnn_benchmark.inference INFO: Total run time: 0:00:19.568536 (0.14495212060433846 s / img per device, on 1 devices)
2020-09-07 00:12:00,587 maskrcnn_benchmark.inference INFO: Total run time: 0:00:19.568536 (0.14495212060433846 s / img per device, on 1 devices)
2020-09-07 00:12:00,592 maskrcnn_benchmark.inference INFO: Model inference time: 0:00:08.359817 (0.06192456881205241 s / img per device, on 1 devices)
2020-09-07 00:12:00,592 maskrcnn_benchmark.inference INFO: Model inference time: 0:00:08.359817 (0.06192456881205241 s / img per device, on 1 devices)
2020-09-07 00:12:00,592 maskrcnn_benchmark.inference INFO: Model inference time: 0:00:08.359817 (0.06192456881205241 s


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x2afc74d22e48>>
Traceback (most recent call last):
  File "/home/bsb2144/.conda/envs/directpose/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/home/bsb2144/.conda/envs/directpose/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/home/bsb2144/.conda/envs/directpose/lib/python3.6/multiprocessing/process.py", line 124, in join
    res = self._popen.wait(timeout)
  File "/home/bsb2144/.conda/envs/directpose/lib/python3.6/multiprocessing/popen_fork.py", line 50, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/home/bsb2144/.conda/envs/directpose/lib/python3.6/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


DONE (t=390.43s)
creating index...
index created!
init bee
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.84s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Averag

In [2]:
dataset_name

'bee_train_cocostyle'

In [3]:
#     NCCL_P2P_DISABLE=1 python -m torch.distributed.launch \
#         --nproc_per_node=8 \
#         --master_port=$((RANDOM + 10000)) \
#         tools/test_net.py \
#         --config-file configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml \
#         DATALOADER.NUM_WORKERS 2 \
#         OUTPUT_DIR training_dir/fcos_kps_ms_training_R_50_FPN_1x \
#         MODEL.WEIGHT fcos_kps_ms_training_R_50_FPN_1x.pth
    
# parser = argparse.ArgumentParser(description="PyTorch Object Detection Inference")
# parser.add_argument(
#     "--config-file",
#     default="/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml",
#     metavar="FILE",
#     help="path to config file",
# )
# parser.add_argument("--local_rank", type=int, default=0)
# parser.add_argument(
#     "opts",
#     help="Modify config options using the command-line",
#     default=None,
#     nargs=argparse.REMAINDER,
# )

# args = parser.parse_args()

num_gpus = int(os.environ["WORLD_SIZE"]) if "WORLD_SIZE" in os.environ else 1
distributed = num_gpus > 1

if distributed:
    torch.cuda.set_device(0)
    torch.distributed.init_process_group(
        backend="nccl", init_method="env://"
    )
    synchronize()

cfg.merge_from_file("/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml")
cfg.merge_from_list(['DATALOADER.NUM_WORKERS', '2', 'OUTPUT_DIR', 'training_dir/fcos_kps_ms_training_R_50_FPN_1x', 'MODEL.WEIGHT', 'fcos_kps_ms_training_R_50_FPN_1x.pth'])
cfg.freeze()

In [4]:
save_dir = ""
logger = setup_logger("maskrcnn_benchmark", save_dir, get_rank())
logger.info("Using {} GPUs".format(num_gpus))
logger.info(cfg)

logger.info("Collecting env info (might take some time)")
logger.info("\n" + collect_env_info())

model = build_detection_model(cfg)
model.to(cfg.MODEL.DEVICE)

output_dir = cfg.OUTPUT_DIR
checkpointer = DetectronCheckpointer(cfg, model, save_dir=output_dir)
_ = checkpointer.load(cfg.MODEL.WEIGHT)

iou_types = ("bbox",)
if cfg.MODEL.MASK_ON:
    iou_types = iou_types + ("segm",)
if cfg.MODEL.KEYPOINT_ON:
    iou_types = iou_types + ("keypoints",)
output_folders = [None] * len(cfg.DATASETS.TEST)
dataset_names = cfg.DATASETS.TEST

2020-09-06 16:04:01,376 maskrcnn_benchmark INFO: Using 1 GPUs
2020-09-06 16:04:01,376 maskrcnn_benchmark INFO: Using 1 GPUs
2020-09-06 16:04:01,377 maskrcnn_benchmark INFO: AMP_VERBOSE: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  NUM_WORKERS: 2
  SIZE_DIVISIBILITY: 32
DATASETS:
  TEST: ('keypoints_coco_2014_minival',)
  TRAIN: ('keypoints_coco_2014_train', 'keypoints_coco_2014_valminusminival')
DATATYPE: bee
DTYPE: float32
INPUT:
  BRIGHTNESS: 0.0
  CONTRAST: 0.0
  CROP_SIZE: 0
  FLIP_PROB_TRAIN: 0.5
  HUE: 0.0
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_RANGE_TRAIN: (-1, -1)
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  PIXEL_MEAN: [102.9801, 115.9465, 122.7717]
  PIXEL_STD: [1.0, 1.0, 1.0]
  SATURATION: 0.0
  TO_BGR255: True
MODEL:
  BACKBONE:
    CONV_BODY: R-50-FPN-RETINANET
    FREEZE_CONV_BODY_AT: 2
    USE_GN: False
  CLS_AGNOSTIC_BBOX_REG: False
  DEVICE: cuda
  FBNET:
    ARCH: default
    ARCH_DEF: 
    BN_TYPE: bn
    DET_HEAD_BLOC

2020-09-06 16:04:01,382 maskrcnn_benchmark INFO: Collecting env info (might take some time)
2020-09-06 16:04:01,382 maskrcnn_benchmark INFO: Collecting env info (might take some time)
2020-09-06 16:04:05,412 maskrcnn_benchmark INFO: 
PyTorch version: 1.1.0
Is debug build: No
CUDA used to build PyTorch: 9.0.176

OS: CentOS Linux 7 (Core)
GCC version: (GCC) 5.2.0
CMake version: Could not collect

Python version: 3.6
Is CUDA available: Yes
CUDA runtime version: 10.1.168
GPU models and configuration: GPU 0: GeForce GTX 1080 Ti
Nvidia driver version: 418.87.00
cuDNN version: Could not collect

Versions of relevant libraries:
[pip3] numpy==1.19.1
[pip3] torch==1.1.0
[pip3] torchvision==0.2.2
[conda] blas                      1.0                         mkl  
[conda] mkl                       2020.2                      256  
[conda] mkl-service               2.3.0            py36he904b0f_0  
[conda] mkl_fft                   1.1.0            py36h23d657b_0  
[conda] mkl_random               

2020-09-06 16:04:06,271 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.0.bn2.running_mean                 loaded from backbone.body.layer1.0.bn2.running_mean                 of shape (64,)
2020-09-06 16:04:06,271 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.0.bn2.running_mean                 loaded from backbone.body.layer1.0.bn2.running_mean                 of shape (64,)
2020-09-06 16:04:06,272 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.0.bn2.running_var                  loaded from backbone.body.layer1.0.bn2.running_var                  of shape (64,)
2020-09-06 16:04:06,272 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.0.bn2.running_var                  loaded from backbone.body.layer1.0.bn2.running_var                  of shape (64,)
2020-09-06 16:04:06,273 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.0.bn2.weight                       loaded from b

2020-09-06 16:04:06,298 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.1.bn1.running_mean                 loaded from backbone.body.layer1.1.bn1.running_mean                 of shape (64,)
2020-09-06 16:04:06,298 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.1.bn1.running_mean                 loaded from backbone.body.layer1.1.bn1.running_mean                 of shape (64,)
2020-09-06 16:04:06,299 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.1.bn1.running_var                  loaded from backbone.body.layer1.1.bn1.running_var                  of shape (64,)
2020-09-06 16:04:06,299 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.1.bn1.running_var                  loaded from backbone.body.layer1.1.bn1.running_var                  of shape (64,)
2020-09-06 16:04:06,305 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.1.bn1.weight                       loaded from b

2020-09-06 16:04:06,337 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.2.bn1.running_var                  loaded from backbone.body.layer1.2.bn1.running_var                  of shape (64,)
2020-09-06 16:04:06,338 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.2.bn1.weight                       loaded from backbone.body.layer1.2.bn1.weight                       of shape (64,)
2020-09-06 16:04:06,338 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.2.bn1.weight                       loaded from backbone.body.layer1.2.bn1.weight                       of shape (64,)
2020-09-06 16:04:06,339 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.2.bn2.bias                         loaded from backbone.body.layer1.2.bn2.bias                         of shape (64,)
2020-09-06 16:04:06,339 maskrcnn_benchmark.utils.model_serialization INFO: backbone.body.layer1.2.bn2.bias                         loaded from b

...

2020-09-06 16:04:06,892 maskrcnn_benchmark.utils.model_serialization INFO: backbone.fpn.top_blocks.p6.weight                       loaded from backbone.fpn.top_blocks.p6.weight                       of shape (256, 256, 3, 3)
2020-09-06 16:04:06,893 maskrcnn_benchmark.utils.model_serialization INFO: backbone.fpn.top_blocks.p7.bias                         loaded from backbone.fpn.top_blocks.p7.bias                         of shape (256,)
2020-09-06 16:04:06,893 maskrcnn_benchmark.utils.model_serialization INFO: backbone.fpn.top_blocks.p7.bias                         loaded from backbone.fpn.top_blocks.p7.bias                         of shape (256,)
2020-09-06 16:04:06,895 maskrcnn_benchmark.utils.model_serialization INFO: backbone.fpn.top_blocks.p7.weight                       loaded from backbone.fpn.top_blocks.p7.weight                       of shape (256, 256, 3, 3)
2020-09-06 16:04:06,895 maskrcnn_benchmark.utils.model_serialization INFO: backbone.fpn.top_blocks.p7.weight            

2020-09-06 16:04:06,919 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.cls_logits.bias                                loaded from rpn.head.cls_logits.bias                                of shape (1,)
2020-09-06 16:04:06,920 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.cls_logits.weight                              loaded from rpn.head.cls_logits.weight                              of shape (1, 256, 3, 3)
2020-09-06 16:04:06,920 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.cls_logits.weight                              loaded from rpn.head.cls_logits.weight                              of shape (1, 256, 3, 3)
2020-09-06 16:04:06,922 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.cls_tower.0.bias                               loaded from rpn.head.cls_tower.0.bias                               of shape (256,)
2020-09-06 16:04:06,922 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.cls_tower.0.bias                          

2020-09-06 16:04:06,946 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.kps_bases.weight                               loaded from rpn.head.kps_bases.weight                               of shape (18, 64, 3, 3)
2020-09-06 16:04:06,947 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.kps_offsets.0.kps_offsets.bias                 loaded from rpn.head.kps_offsets.0.kps_offsets.bias                 of shape (10,)
2020-09-06 16:04:06,947 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.kps_offsets.0.kps_offsets.bias                 loaded from rpn.head.kps_offsets.0.kps_offsets.bias                 of shape (10,)
2020-09-06 16:04:06,949 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.kps_offsets.0.kps_offsets.weight               loaded from rpn.head.kps_offsets.0.kps_offsets.weight               of shape (10, 64, 1, 1)
2020-09-06 16:04:06,949 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.kps_offsets.0.kps_offsets.weight          

2020-09-06 16:04:06,973 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.sample_features_conv.0.bias                    loaded from rpn.head.sample_features_conv.0.bias                    of shape (576,)
2020-09-06 16:04:06,974 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.sample_features_conv.0.weight                  loaded from rpn.head.sample_features_conv.0.weight                  of shape (576, 64, 3, 3)
2020-09-06 16:04:06,974 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.sample_features_conv.0.weight                  loaded from rpn.head.sample_features_conv.0.weight                  of shape (576, 64, 3, 3)
2020-09-06 16:04:06,976 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.sample_features_conv.1.bias                    loaded from rpn.head.sample_features_conv.1.bias                    of shape (576,)
2020-09-06 16:04:06,976 maskrcnn_benchmark.utils.model_serialization INFO: rpn.head.sample_features_conv.1.bias           

RuntimeError: Error(s) in loading state_dict for GeneralizedRCNN:
	size mismatch for rpn.head.kps_tower.0.weight: copying a param with shape torch.Size([576, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([64, 256, 3, 3]).
	size mismatch for rpn.head.kps_tower.0.bias: copying a param with shape torch.Size([576]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for rpn.head.kps_tower.1.weight: copying a param with shape torch.Size([576]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for rpn.head.kps_tower.1.bias: copying a param with shape torch.Size([576]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for rpn.head.kps_tower.3.weight: copying a param with shape torch.Size([576, 64, 3, 3]) from checkpoint, the shape in current model is torch.Size([64, 64, 3, 3]).
	size mismatch for rpn.head.kps_tower.3.bias: copying a param with shape torch.Size([576]) from checkpoint

In [ ]:
if cfg.OUTPUT_DIR:
    for idx, dataset_name in enumerate(dataset_names):
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference", dataset_name)
        mkdir(output_folder)
        output_folders[idx] = output_folder
data_loaders_val = make_data_loader(cfg, is_train=False, is_distributed=distributed)


In [ ]:
# for output_folder, dataset_name, data_loader_val in zip(output_folders, dataset_names, data_loaders_val):
#     inference(
#         model,
#         data_loader_val,
#         dataset_name=dataset_name,
#         iou_types=iou_types,
#         box_only=False if cfg.MODEL.FCOS_ON or cfg.MODEL.RETINANET_ON else cfg.MODEL.RPN_ONLY,
#         device=cfg.MODEL.DEVICE,
#         expected_results=cfg.TEST.EXPECTED_RESULTS,
#         expected_results_sigma_tol=cfg.TEST.EXPECTED_RESULTS_SIGMA_TOL,
#         output_folder=output_folder,
#     )
#     synchronize()

In [ ]:
for output_folder, dataset_name, data_loader_val in zip(output_folders, dataset_names, data_loaders_val):
    print(output_folder,dataset_name,data_loader_val)

box_only=False if cfg.MODEL.FCOS_ON or cfg.MODEL.RETINANET_ON else cfg.MODEL.RPN_ONLY
device=cfg.MODEL.DEVICE
expected_results=cfg.TEST.EXPECTED_RESULTS
expected_results_sigma_tol=cfg.TEST.EXPECTED_RESULTS_SIGMA_TOL
data_loader = data_loader_val

In [ ]:
import logging
import time
import os

import torch
from tqdm import tqdm

from maskrcnn_benchmark.data.datasets.evaluation import evaluate
from maskrcnn_benchmark.utils.comm import get_world_size
from maskrcnn_benchmark.utils.timer import Timer, get_time_str
from maskrcnn_benchmark.engine.inference import compute_on_dataset

# convert to a torch.device for efficiency
device = torch.device(device)
num_devices = get_world_size()
logger = logging.getLogger("maskrcnn_benchmark.inference")
dataset = data_loader.dataset
logger.info("Start evaluation on {} dataset({} images).".format(dataset_name, len(dataset)))
total_timer = Timer()
inference_timer = Timer()
total_timer.tic()

In [ ]:
timer=inference_timer
model.eval()
results_dict = {}
cpu_device = torch.device("cpu")
c = 0
for _, batch in enumerate(tqdm(data_loader)):
    if c>0:
        break
    c+=1
    

In [ ]:
images, targets, image_ids = batch
images = images.to(device)
img = images.tensors.cpu().numpy()
print(img.shape)

In [ ]:
# with torch.no_grad():
#     if timer:
#         timer.tic()
    output = model(images)
#     if timer:
#         torch.cuda.synchronize()
#         timer.toc()
#     output = [o.to(cpu_device) for o in output]
# results_dict.update(
#     {img_id: result for img_id, result in zip(image_ids, output)}
# )


# return results_dict

In [ ]:
# predictions = compute_on_dataset(model, data_loader, device, inference_timer)

# # wait for all processes to complete before measuring the time
# synchronize()
# total_time = total_timer.toc()
# total_time_str = get_time_str(total_time)
# logger.info(
#     "Total run time: {} ({} s / img per device, on {} devices)".format(
#         total_time_str, total_time * num_devices / len(dataset), num_devices
#     )
# )
# total_infer_time = get_time_str(inference_timer.total_time)
# logger.info(
#     "Model inference time: {} ({} s / img per device, on {} devices)".format(
#         total_infer_time,
#         inference_timer.total_time * num_devices / len(dataset),
#         num_devices,
#     )
# )

# predictions = _accumulate_predictions_from_multiple_gpus(predictions)
# if not is_main_process():
#     return

# if output_folder:
#     torch.save(predictions, os.path.join(output_folder, "predictions.pth"))

# extra_args = dict(
#     box_only=box_only,
#     iou_types=iou_types,
#     expected_results=expected_results,
#     expected_results_sigma_tol=expected_results_sigma_tol,
# )

# return evaluate(dataset=dataset,
#                 predictions=predictions,
#                 output_folder=output_folder,
#                 **extra_args)

In [ ]:
import random

aa = 1
a = random.choice(aa)

if isinstance(self.min_size[0], int):
    size = random.choice(self.min_size)
else:
        size = random.choice(self.min_size[0])